Models. https://gist.github.com/ElmanSafarov1960/2f6a1933d43b17e3795964ea04c89271

In [ ]:
X = df.drop(columns=['churn', 'id'])
y = df['churn']

# Проверка бесконечностей
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.median())

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Перевірка результату

print("\n***Розміри наборів:***")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)


***Розміри наборів:***
X_train: (57819, 10)
X_test: (14455, 10)
y_train: (57819,)
y_test: (14455,)


Models. 3.1. Вибір та ініціалізація алгоритмів машинного навчання. Для розв’язання задачі прогнозування відтоку клієнтів телекомунікаційної компанії було обрано декілька алгоритмів машинного навчання, які належать до різних класів моделей. Такий підхід дозволяє провести порівняльний аналіз їх ефективності та обрати найбільш придатну модель для практичного застосування. У межах дослідження використано логістичну регресію, Random Forest, Gradient Boosting та штучну нейронну мережу типу Multilayer Perceptron (MLP). Логістична регресія застосовується як базова лінійна модель завдяки простоті реалізації та високій інтерпретованості результатів. Random Forest та Gradient Boosting належать до ансамблевих методів і дозволяють ефективно моделювати нелінійні залежності між ознаками. Нейронна мережа MLP була використана як додаткова модель для аналізу складних нелінійних взаємозв’язків між характеристиками клієнтів.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

models = {
    "Logistic Regression": LogisticRegression(
        max_iter=1000,
        class_weight="balanced",
        random_state=42
    ),

    "Random Forest": RandomForestClassifier(
        n_estimators=200,
        random_state=42,
        class_weight="balanced"
    ),

    "Gradient Boosting": GradientBoostingClassifier(
        random_state=42
    ),

    "Neural Network (MLP)": MLPClassifier(
        hidden_layer_sizes=(64, 32),
        activation='relu',
        solver='adam',
        alpha=0.001,
        max_iter=500,
        random_state=42
    )
}

3.2. Базове навчання та первинна оцінка моделей Перед навчанням моделей набір даних було поділено на тренувальну та тестову вибірки у співвідношенні 80% до 20% із застосуванням стратифікації за цільовою змінною, що забезпечує збереження пропорцій класів. Числові ознаки були стандартизовані з використанням методу StandardScaler, що є обов’язковим для коректної роботи логістичної регресії та нейронної мережі. Базове навчання кожної моделі проводилося на тренувальному наборі даних, після чого здійснювалася оцінка якості класифікації за метрикою Accuracy на тестовому наборі. Отримані результати показали, що ансамблеві методи та нейронна мережа демонструють вищу якість прогнозування порівняно з лінійною моделлю.

In [ ]:
results = {}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    results[name] = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {results[name]:.4f}")

Logistic Regression Accuracy: 0.8785
Random Forest Accuracy: 0.9404
Gradient Boosting Accuracy: 0.9373
Neural Network (MLP) Accuracy: 0.9333


 3.3. Крос-валідація та підбір гіперпараметрів. З метою підвищення якості прогнозування було виконано підбір оптимальних гіперпараметрів для моделі Random Forest з використанням методу Grid Search у поєднанні з п’ятикратною крос-валідацією. Такий підхід дозволяє зменшити ризик перенавчання та отримати більш стабільну оцінку ефективності моделі. Як основну метрику оптимізації було обрано F1-score, оскільки вона враховує баланс між точністю та повнотою класифікації, що є особливо важливим для задачі прогнозування відтоку клієнтів. Нейронна мережа MLP використовувалася без додаткового підбору гіперпараметрів та слугувала порівняльною моделлю. GridSearch выполняется для лучшей ансамблевой модели — Random Forest
 3.4. Фінальна оцінка якості моделі Фінальна оцінка якості була проведена для моделі Random Forest з оптимальними гіперпараметрами на незалежному тестовому наборі даних. Для комплексного аналізу ефективності використано такі метрики, як Accuracy, Precision, Recall та F1-score. Застосування кількох метрик дозволяє більш повно оцінити здатність моделі правильно ідентифікувати клієнтів, схильних до відтоку, та мінімізувати кількість помилкових прогнозів. Отримані результати підтверджують доцільність використання ансамблевих методів для вирішення даної задачі.

In [ ]:
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [None, 5, 10],
    "min_samples_split": [2, 5]
}

rf = RandomForestClassifier(
    random_state=42,
    class_weight="balanced"
)

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring="f1",
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)

best_model = grid_search.best_estimator_

Best parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}


In [ ]:
y_pred_final = best_model.predict(X_test)

print("Final model metrics:")
print("Accuracy:", accuracy_score(y_test, y_pred_final))
print("Precision:", precision_score(y_test, y_pred_final))
print("Recall:", recall_score(y_test, y_pred_final))
print("F1-score:", f1_score(y_test, y_pred_final))

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred_final))

Final model metrics:
Accuracy: 0.9406433759944656
Precision: 0.9571720787522373
Recall: 0.9347066167290886
F1-score: 0.9458059626073775

Classification Report:

              precision    recall  f1-score   support

           0       0.92      0.95      0.93      6445
           1       0.96      0.93      0.95      8010

    accuracy                           0.94     14455
   macro avg       0.94      0.94      0.94     14455
weighted avg       0.94      0.94      0.94     14455



In [ ]:
y_proba = best_model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_proba)
print("ROC AUC:", auc)

fpr, tpr, _ = roc_curve(y_test, y_proba)

plt.plot(fpr, tpr)
plt.plot([0,1],[0,1],'--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

 ROC AUC: 0.980840251082578


3.5. Аналіз важливості ознак Заключним етапом дослідження став аналіз важливості ознак для фінальної моделі Random Forest. Для цього було використано вбудований механізм оцінки важливості ознак, який дозволяє визначити внесок кожної характеристики у процес прийняття рішення моделлю. Результати аналізу були візуалізовані у вигляді стовпчикової діаграми, що надало можливість виділити найбільш значущі фактори впливу на відтік клієнтів. Отримана інформація може бути використана для формування бізнес-рішень, спрямованих на підвищення рівня утримання клієнтів.

In [ ]:
feature_importance = pd.Series(
    best_model.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

plt.figure(figsize=(10,6))
feature_importance.head(10).plot(kind="bar")
plt.title("Feature Importance (Random Forest)")
plt.ylabel("Importance")
plt.show()